##### 베이즈 정리로 텍스트 분류하기 

In [ ]:
# 베이지안 필터(Bayesian filter) :  학습을 많이 시키면 시킬수록 필터의 분류 능력이 오름
#                                 사용자가 따로 키워드 지정을 하지 않아도 스팸 메일을 분류

# 나이브 베이즈 분류(Niive B:yes classifier) 알고리즘 사용

# 베이즈 정리 :  A라는 사건이 B에 속하는지를 판단
#   => 나이브 베이즈 분류는 텍스트 내부에서 단어 출현 비율을 조사하여 해당 텍스트를 어떤 카테고리로 분류하는 것이 적합한지 조사

In [3]:
import math, sys
from konlpy.tag import Twitter, Okt

class BayesianFilter :
    ''' 베이지안 필터'''
    def __init__(self) :
        self.words = set()          # 출현 단어 기록
        self.word_dict = {}         # 카테고리마다의 출현 횟수 기록
        self.category_dict = {}     # 카테고리 출현 횟수 기록   
    
    ######################### 연산 함수 #############################
    
    # 예측하기
    def predict(self, text) :
        best_category = None
        max_score = -sys.maxsize
        words = self.split(text)        # 텍스트를 형태소 단위로 분해
        score_list = []
        for category in self.category_dict.keys() :     # 카테고리가 카테고리 리스트에 있다면 점수 매기기
            score = self.score(words, category)     
            score_list.append((category, score))
            if score > max_score :
                max_score = score
                best_category = category
        return best_category, score_list
        
    ######################### 형태소 분해 및 계산 함수 #############################    
    
    ### 텍스트 분류 및 학습 - 형태소 단위로 나눈 텍스트를 word_dict와 category_dict에 저장
    def fit(self, text, category) :
        word_list = self.split(text)
        for word in word_list :
            self.inc_word(word, category)
        self.inc_category(category)
    
    # 형태소 단위로 분해하기
    def split(self, text) :
        results = []
        twitter = Okt()
        
        # 단어 기본형 적용
        malist = twitter.pos(text, norm=True, stem=True)
        for word in malist :
            if not word[1] in ['Josa','Eomi', 'Punctuation'] : # 어미/조사/구두점 제외
                results.append(word[0])
        return results
    
    # 단어와 카테고리의 출현 횟수 세기  (단어와 카테고리를 입력받아 word_dict에 저장)
    def inc_word(self, word, category) :
        # 단어를 카테고리에 추가
        if not category in self.word_dict :
            self.word_dict[category] = {}
        if not word in self.word_dict[category] :
            self.word_dict[category][word] = 0
        self.word_dict[category][word] += 1     # 값이 없을 경우 0을 나눌 수 없기 때문에 더해줌 (어차피 효과가 미미함)
        self.words.add(word)
    
    # 카테고리 수집  (카테고리를 입력받아 category_dict에 저장)
    def inc_category(self, category) :
        if not category in self.category_dict :
            self.category_dict[category] = 0
        self.category_dict[category] += 1    
    
    ### 단어 리스트에 점수 매기기
    def score(self, words, category) :
        score = math.log(self.category_prob(category))
        for word in words :
            score += math.log(self.word_prob(word, category))
        return score
        
    # 특정 카테고리의 비율 계산
    def category_prob(self, category):
        sum_categories = sum(self.category_dict.values())
        category_v = self.category_dict[category]
        return category_v / sum_categories
    
    # 카테고리 내부의 단어 출현 비율 계산
    def word_prob(self, word, category):
        n = self.get_word_count(word, category) + 1
        d = sum(self.word_dict[category].values()) + len(self.words)
        return n / d
        
    # 카테고리 내부의 단어 출현 횟수 구하기
    def get_word_count(self, word, category):
        if word in self.word_dict[category]:
            return self.word_dict[category][word]
        else:
            return 0        

In [6]:
# from bayes import BayesianFilter      => 베이지안필터 모듈

bf = BayesianFilter()    # 클래스 호출(객체 생성)

# 텍스트 학습
bf.fit("파격 세일 - 오늘까지만 30% 할인", "광고")
bf.fit("쿠폰 선물 & 무료 배송", "광고")
bf.fit("현대 백화점 세일", "광고")
bf.fit("봄과 함께 찾아온 따뜻한 신제품 소식", "광고")
bf.fit("인기 제품 기간 한정 세일", "광고")
bf.fit("오늘 일정 확인", "중요")
bf.fit("프로젝트 진행 상황 보고", "중요")
bf.fit("계약 잘 부탁드립니다", "중요")
bf.fit("회의 일정이 등록되었습니다.","중요")
bf.fit("오늘 일정이 없습니다.","중요")
bf.fit("재고 확인 부탁드립니다.","중요")

# 예측
pre, scorelist = bf.predict("재고 정리 할인, 무료 배송")
print('결과 =',pre)
print(scorelist)

결과 = 광고
[('광고', -19.65728952881656), ('중요', -20.215203675742465)]


##### MLP로 텍스트 분류하기 

In [ ]:
# 다층 퍼셉트론(Multi Layer Perceptron, MLP) : 입력층과 출력층 사이에 각각 전체 결합하는 은닉층을 넣은 뉴럴 네트워크
#                                            텍스트 데이터를 숫자로 표현할 수 있는 벡터로 변환

### 텍스트 분류 과정
# (1) 텍스트에서 불필요한 품사를 제거한다.
# (2) 사전을 기반으로 단어를 숫자로 변환한다.
# (3) 파일 내부의 단어 출현 비율을 계산한다.
# (4) 데이터를 학습시킨다.
# (5) 테스트 데이터를 넣어 성공률을 확인한다.

In [12]:
#### 텍스트를 단어로 정제 및 단어를 숫자로 변환하는 코드

import os, glob, json

root_dir = 'data/newstext'
dic_file = root_dir+'/word-dic.json'
data_file = root_dir+'/data.json'
data_file_min = root_dir+'/data-mini.json'

# 데이터 어구를 자르고 ID로 변환하는 함수 - 단어의 ID리스트를 리턴
word_dic = {'_MAX' : 0}
def text_to_ids(text) :
    text = text.strip()
    words = text.split(" ")
    result = []
    for n in words:
        n = n.strip()
        if n == '' : continue
        if not n in word_dic :
            wid = word_dic[n] = word_dic['_MAX']        # 단어에 ID값 부여
            word_dic['_MAX'] += 1
        else : wid = word_dic[n]
        result.append(wid)
    return result     

# 딕셔너리(word_dic)에 단어를 저장하는 함수 - file_to_ids()에서 파일 읽고 text_to_ids()에서 ID값 부여
def register_dic() :
    files = glob.glob(root_dir+'/*/*.wakati', recursive=True)
    for i in files :
        file_to_ids(i)
        
# 파일의 데이터에서 고정 길이 배열 리턴하기 (ID로 변환)
def file_to_ids(fname) :
    with open(fname, 'r', encoding='utf-8') as f :
        text = f.read()
        return text_to_ids(text)        
    
  
        
# 파일 내부의 단어 카운트
def count_file_freq(fname) :
    cnt = [0 for n in range(word_dic['_MAX'])]      # 단어 ID의 개수만큼 0을 채운 리스트 생성
    with open(fname, 'r', encoding='utf-8') as f :
        text = f.read().strip()
        ids = text_to_ids(text)     # 해당 text의 id값을 가져와서 count -> 단어 개수 카운트
        for wid in ids :
            cnt[wid] += 1
    return cnt

# 카테고리마다 파일 읽어오는 함수
def count_freq(limit = 0) :
    X = []      # 단어 개수 카운트 리스트
    Y = []      # 카운트 리스트의 인덱스 역할을 하는 리스트
    max_words = word_dic['_MAX']
    cat_names = []
    for cat in os.listdir(root_dir) :
        cat_dir = root_dir + '/' + cat
        if not os.path.isdir(cat_dir) : continue
        cat_idx = len(cat_names)
        cat_names.append(cat)
        files = glob.glob(cat_dir + '/*.wakati') 
        i = 0
        for path in files :
            print(path)
            cnt = count_file_freq(path)
            X.append(cnt)
            Y.append(cat_idx)
            if limit > 0 :  # limit에 입력값이 있으면 limit까지만 반복
                if i > limit : break    
                i += 1
    return X, Y

# 단어 딕셔너리 만들기
if os.path.exists(dic_file) :
    word_dic = json.load(open(dic_file))
else : 
    register_dic()
    json.dump(word_dic, open(dic_file, 'w', encoding='utf-8'))

# 벡터를 파일로 출력하기
# 테스트 목적의 소규모 데이터 만들기
X, Y = count_freq(20)
json.dump({'X' : X, 'Y' : Y}, open(data_file_min, 'w', encoding='utf-8'))

# 전체 데이터를 기반으로 만들기
X, Y = count_freq()
json.dump({'X': X, 'Y':Y}, open(data_file, 'w', encoding='utf-8'))
print('ok')



data/newstext/100\1.txt.wakati
data/newstext/100\10.txt.wakati
data/newstext/100\100.txt.wakati
data/newstext/100\1000.txt.wakati
data/newstext/100\1001.txt.wakati
data/newstext/100\1002.txt.wakati
data/newstext/100\1003.txt.wakati
data/newstext/100\1004.txt.wakati
data/newstext/100\1005.txt.wakati
data/newstext/100\1006.txt.wakati
data/newstext/100\1007.txt.wakati
data/newstext/100\1008.txt.wakati
data/newstext/100\1009.txt.wakati
data/newstext/100\101.txt.wakati
data/newstext/100\1010.txt.wakati
data/newstext/100\1011.txt.wakati
data/newstext/100\1012.txt.wakati
data/newstext/100\1013.txt.wakati
data/newstext/100\1014.txt.wakati
data/newstext/100\1015.txt.wakati
data/newstext/100\1016.txt.wakati
data/newstext/100\1017.txt.wakati
data/newstext/101\1.txt.wakati
data/newstext/101\10.txt.wakati
data/newstext/101\100.txt.wakati
data/newstext/101\1000.txt.wakati
data/newstext/101\1001.txt.wakati
data/newstext/101\1002.txt.wakati
data/newstext/101\1003.txt.wakati
data/newstext/101\1004.txt.

##### Keras를 이용한 MLP 텍스트 분류 

In [32]:
### 필요한 모듈 import
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import model_selection, metrics
import json

max_words = 77238   # word-dic.json 파일 참고
nb_classes = 6      # 6개의 카테고리
batch_size = 64     # 한 번에 학습할 단어 개수
epochs = 20       # 회차

## MLP 모델 생성하는 함수
def build_model() :
    model = Sequential()
    model.add(Dense(512, input_shape = (max_words,)))   # 단어 개수만큼 행을 가진 행렬로 입력, 출력값 512개
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(nb_classes))
    model.add(Activation('softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

## 데이터 읽어오기
# data = json.load(open('data/newstext/data-mini.json'))    # 정답률 = 0.7575757575757576
data = json.load(open('data/newstext/data.json')) 
X = data["X"]   # 텍스트 데이터
Y = data["Y"]   # 카테고리 데이터

In [33]:
## 학습하기
X_train, X_test, Y_train, Y_test = train_test_split(X,Y)

Y_train = np_utils.to_categorical(Y_train, num_classes=nb_classes)      # Y의 숫자를 다시 카테고리화
X_train = np.asarray(X_train)

print(len(X_train),len(Y_train))

model = KerasClassifier(build_fn=build_model, epochs=epochs, batch_size=batch_size)
model.fit(X_train, Y_train)

13374 13374


C:\Users\tjoeun709\AppData\Local\Temp\ipykernel_7244\96352554.py:9: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=build_model, epochs=epochs, batch_size=batch_size)


Epoch 1/20
209/209 [==============================] - 44s 161ms/step - loss: 0.4629 - accuracy: 0.8718
Epoch 2/20
209/209 [==============================] - 33s 157ms/step - loss: 0.1715 - accuracy: 0.9554
Epoch 3/20
209/209 [==============================] - 33s 156ms/step - loss: 0.1025 - accuracy: 0.9747
Epoch 4/20
209/209 [==============================] - 32s 154ms/step - loss: 0.0954 - accuracy: 0.9800
Epoch 5/20
209/209 [==============================] - 32s 154ms/step - loss: 0.0755 - accuracy: 0.9838
Epoch 6/20
209/209 [==============================] - 32s 154ms/step - loss: 0.0763 - accuracy: 0.9852
Epoch 7/20
209/209 [==============================] - 32s 155ms/step - loss: 0.0785 - accuracy: 0.9847
Epoch 8/20
209/209 [==============================] - 32s 152ms/step - loss: 0.0788 - accuracy: 0.9871
Epoch 9/20
209/209 [==============================] - 32s 151ms/step - loss: 0.0846 - accuracy: 0.9870
Epoch 10/20
209/209 [==============================] - 32s 152ms/step - l

In [34]:
## 예측하기
y = model.predict(X_test)
ac_score = metrics.accuracy_score(Y_test, y)
cl_report = metrics.classification_report(Y_test, y)
print('정답률 =', ac_score)
print('리포트 =\n', cl_report)

140/140 [==============================] - 7s 27ms/step
정답률 = 0.8869701726844584
리포트 =
               precision    recall  f1-score   support

           0       0.94      0.91      0.93      1013
           1       0.83      0.93      0.88       858
           2       0.88      0.85      0.86       500
           3       0.85      0.92      0.88       686
           4       0.89      0.88      0.89       889
           5       0.94      0.77      0.85       513

    accuracy                           0.89      4459
   macro avg       0.89      0.88      0.88      4459
weighted avg       0.89      0.89      0.89      4459

